In [ ]:
! pip install mysql
! pip install mysql-connector-python
! pip install yfinance
! pip install -U numpy
! pip install gspread-dataframe
! pip install schedule

In [7]:
import numpy as np
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
import json
import csv
import datetime
import numpy as np
import mysql.connector
from functools import lru_cache

import gspread
from google.oauth2.service_account import Credentials
import gspread_dataframe as gd
from functools import lru_cache
import yfinance as yf 

In [8]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = service_account.Credentials.from_service_account_file('keys.json', scopes=SCOPES)

# The ID of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1KL338RavMblh2XdkIhFJ5QpLIn649E-ZQ1hIr9Rm124'

In [9]:
def get_data_db():
    mydb = mysql.connector.connect(
        host="xxx",
        user="xxx",
        password="xxx",
        database="xxx",
        port=xxx)

    # SQL query to retrieve distinct stock symbols
    query = """SELECT DISTINCT SymbolName FROM sm_stocks WHERE MarketName = 'SET' OR MarketName = 'MAI';"""

    mycursor = mydb.cursor()
    mycursor.execute(query)

    # Retrieve column names
    column_names = [desc[0] for desc in mycursor.description]

    myresult = mycursor.fetchall()

    df = pd.DataFrame(myresult,columns=column_names)

    return df   
    
def download_stock_data():
    df = get_data_db().copy()
    # Set the date range for downloading all data
    start_date = "2021-07-27"
    #ดาวโหลดข้อมูลเผื่อคิด avg 5 วันก่อนหน้า
    end_date = pd.to_datetime("today").strftime("%Y-%m-%d")

    # List of stock symbols with ".BK" appended
    symbol_names = df['SymbolName'].tolist()
    symbols = [f"{symbol}.BK" for symbol in symbol_names]

    # Initialize an empty DataFrame to store the data
    all_data = pd.DataFrame()

    # Download data for each symbol
    for symbol in symbols:
        # Download data for the current symbol
        data = yf.download(symbol, start=start_date, end=end_date)

        # Ensure that the data is not empty
        if not data.empty:
            # Add a new column 'Symbol' to the data and set it to the current symbol
            data['Symbol'] = symbol
            
            # Append the data to the all_data DataFrame
            all_data = pd.concat([all_data, data])

    # Sort the data by index (date)
    all_data.sort_index(inplace=True)

    all_data.reset_index(inplace=True)

    all_data = pd.DataFrame(all_data)
    
    def format_number(num):
        if isinstance(num, (int, float)):  # ตรวจสอบว่าค่าเป็นตัวเลขหรือไม่
            return "{:,.2f}".format(num)
        else:
            return num

    # ใช้ applymap() เพื่อใช้ฟังก์ชันแบบกำหนดเองในทุกเซลล์
    all_data = all_data.applymap(format_number)

    # ลบลูกน้ำและแทนที่ ',' ด้วย '' ในคอลัมน์ 'Volume' ใน all_data
    all_data['Volume'] = all_data['Volume'].str.replace(',', '', regex=True)

    # แปลงคอลัมน์ 'Volume' เป็นตัวเลขใน all_data
    all_data['Volume'] = pd.to_numeric(all_data['Volume'], errors='coerce')

    all_data = all_data[(all_data['Volume'].astype(float) > 10000000) & (all_data['Close'].astype(float) > 0.5)]
    # สร้าง column gainer เพื่อใช้หา gainer สูงสุด

    # แปลงคอลัมน์ 'Close' และ 'Open' เป็นชนิดข้อมูล float
    all_data['Close'] = all_data['Close'].astype(float)
    all_data['Open'] = all_data['Open'].astype(float)

    all_data = all_data.copy()  # คัดลอก DataFrame filtered_data เพื่อป้องกันการเปลี่ยนแปลงข้อมูลใน DataFrame ต้นฉบับ

    # สร้างคอลัมน์ใหม่ใน DataFrame filtered_data2 โดยคำนวณสูตร (close - open) / open
    all_data['%Change_O2C'] = ((all_data['Close'] - all_data['Open']) / all_data['Open']) * 100 ###

    # เรียงลำดับ DataFrame โดยใช้คอลัมน์ PercentChange
    all_data = all_data.sort_values(by='%Change_O2C', ascending=False)

    return all_data     

def filtered_data():
    filtered_data = download_stock_data().copy()
    filtered_data = filtered_data.sort_values(by=['Symbol', 'Date'])
    dfft = {
        'Date': filtered_data['Date'],
        'Symbol': filtered_data['Symbol'],
        '%Change_C2C': (filtered_data.groupby('Symbol')['Close'].transform(lambda x: (x - x.shift(1)) / x.shift(1) * 100)),
        '%CMPR': (filtered_data.groupby('Symbol')['Volume'].transform(lambda x: x / x.shift(5).mean()) * 100),
        'Volume': filtered_data['Volume'],
        '%Change_O2C': filtered_data['%Change_O2C']
    }

    filtered_data = pd.DataFrame(dfft)

    # เรียงลำดับ DataFrame ตามคอลัมน์ Date และ Symbol
    filtered_data = filtered_data.sort_values(by=['Date', '%Change_O2C'], ascending=[True, False])
    filtered_data = filtered_data[filtered_data['Date'] >= '2021-08-27']

    filtered_data = filtered_data[(filtered_data['%CMPR'] > 100) & (filtered_data['%Change_C2C'] > 0.01)]
    filtered_data['Symbol'] = filtered_data['Symbol'].str.replace('.BK', '')

    return filtered_data  




In [11]:
def unique_symbols():    
    # นับรายการ Symbol โดยไม่ซ้ำชื่อ
    df = filtered_data().copy()
    df = df['Symbol'].unique()
    df = pd.DataFrame(df, columns=['symbols'])

    df['symbols'] = df['symbols'].str.replace('.BK', '')
    df = df['symbols'].tolist()

    return df 

def df_buy_sell():
    df = unique_symbols().copy()
    mydb = mysql.connector.connect(
        host="xxx",
        user="xxx",
        password="xxx",
        database="xxx",
        port=xxx)

    # Create a cursor to execute SQL commands
    mycursor11 = mydb.cursor()

    
    # สร้าง DataFrame เปล่าสำหรับรวบรวมผลลัพธ์
    df_buy_sell = pd.DataFrame()

    # SQL query to retrieve distinct stock symbols
    for symbol_to_query in df:
        query = f"""
        SELECT DATE(DtMinOfRec) AS Date, 
            Symbol, 
            SUM(CASE WHEN Type = 'BUY' THEN Vol ELSE 0 END) AS BUY,
            SUM(CASE WHEN Type = 'SELL' THEN Vol ELSE 0 END) AS SELL
        FROM stocksm_tickmatchs
        WHERE (TIME(DtMinOfRec) BETWEEN '10:00:00' AND '12:30:00'
            OR TIME(DtMinOfRec) BETWEEN '14:30:00' AND '16:45:00')
        AND DtMinOfRec >= '2021-08-27 00:00:00'
        AND Symbol IN ('{symbol_to_query}')
        GROUP BY Date, Symbol
        ;"""
        # Execute the SQL query
        mycursor11.execute(query)

        # Read the query results
        myresult11 = mycursor11.fetchall()

        # Get the column names from the cursor description
        column_names11 = [desc[0] for desc in mycursor11.description]

        # Create a DataFrame using the query results and set column names
        df_symbol = pd.DataFrame(myresult11, columns=column_names11)

        # เพิ่ม DataFrame สำหรับสัญลักษณ์นี้ใน df_buy_sell โดยใช้ pd.concat()
        df_buy_sell = pd.concat([df_buy_sell, df_symbol], ignore_index=True)

    mycursor11.close()

    

    df_buy_sell = pd.DataFrame(df_buy_sell)

    # เปลี่ยนประเภทของคอลัมน์ 'Date' เป็น datetime64[ns]
    df_buy_sell['Date'] = pd.to_datetime(df_buy_sell['Date'])

    return df_buy_sell  

#############################
def filtered_datafinal():
    df = df_buy_sell().copy()
    filtered_datafinal =filtered_data().copy()
    # เริ่มต้นด้วยการสร้างคอลัมน์ใหม่ใน filtered_datafinal สำหรับ BUY และ SELL
    filtered_datafinal['BUY'] = ''
    filtered_datafinal['SELL'] = ''


    # กำหนดฟังก์ชันสำหรับการแสดงผลตัวเลข
    def format_number(num):
        if isinstance(num, (int, float)):  # ตรวจสอบว่าค่าเป็นตัวเลขหรือไม่
            return "{:,.2f}".format(num)
        else:
            return num

    # ในลูป for จะเข้าถึงแต่ละแถวใน filtered_datafinal
    for index, row in filtered_datafinal.iterrows():
        date_symbol_match = df[(df['Date'] == row['Date']) & (df['Symbol'] == row['Symbol'])]
        
        # ถ้ามีข้อมูลที่ตรงกันใน df
        if not date_symbol_match.empty:
            filtered_datafinal.at[index, 'BUY'] = date_symbol_match['BUY'].values[0]
            filtered_datafinal.at[index, 'SELL'] = date_symbol_match['SELL'].values[0]


    filtered_datafinal = filtered_datafinal.applymap(format_number)

    # แปลงคอลัมน์ 'BUY,SELL' 
    filtered_datafinal['BUY'] = pd.to_numeric(filtered_datafinal['BUY'], errors='coerce')
    filtered_datafinal['SELL'] = pd.to_numeric(filtered_datafinal['SELL'], errors='coerce')

    filtered_datafinal = filtered_datafinal.reset_index(drop=True)

    filtered_datafinal = pd.DataFrame(filtered_datafinal)

    # ฟังก์ชันสำหรับคำนวณค่าร้อยละของ BUY ในแต่ละแถว
    def calculate_buy_percentage(row):
        total = row['BUY'] + row['SELL']
        return (row['BUY'] / total) * 100 if total != 0 else 0

    # เลือกรายการที่ BUY มากกว่า 51%
    filtered_datafinal = filtered_datafinal[filtered_datafinal.apply(calculate_buy_percentage, axis=1) > 51]
    filtered_datafinal['%BUY'] = (filtered_datafinal['BUY'] / (filtered_datafinal['BUY'] + filtered_datafinal['SELL'])) * 100


    # เรียงข้อมูลตามคอลัมน์ "Date" และ "Change_O2C"
    filtered_datafinal = filtered_datafinal.sort_values(by=['Date', '%Change_O2C'], ascending=[False, False])

    # แบ่งข้อมูลเป็นกลุ่มตาม "Date" และแสดงข้อมูล 30 แถวแรกในแต่ละกลุ่ม
    filtered_datafinal = filtered_datafinal.groupby('Date').head(30)
    filtered_datafinal = filtered_datafinal.applymap(format_number)
    filtered_datafinal['Date'] = filtered_datafinal['Date'].apply(lambda x: pd.to_datetime(x).date())
    
    


    
    return filtered_datafinal


In [ ]:
import pandas as pd
import gspread
from google.oauth2 import service_account
from datetime import datetime

def run_all():
    # ระบุขอบเขต (scope) ของการเข้าถึง Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    # อ่านข้อมูลการรับรองตัวตนจากไฟล์ "keys.json"
    creds = service_account.Credentials.from_service_account_file(
        'keys.json', scopes=SCOPES)

    # เชื่อมต่อกับ Google Sheets
    client = gspread.authorize(creds)
    
    # รหัสของ Google Spreadsheet ที่คุณต้องการใช้
    SAMPLE_SPREADSHEET_ID = '1KL338RavMblh2XdkIhFJ5QpLIn649E-ZQ1hIr9Rm124'

    # โหลดข้อมูลหุ้นที่คุณต้องการอัปเดต (เรียกใช้ฟังก์ชันหรือเมทอดที่ให้ข้อมูลนี้)
    filtered_data = filtered_datafinal() # ต้องแทนที่ด้วยการโหลดข้อมูลของคุณจากฟังก์ชัน filtered_datafinal()

    # เปิด Google Sheet
    sheet = client.open_by_key(SAMPLE_SPREADSHEET_ID)

    # เลือกชีทที่คุณต้องการอัปเดต (ใช้ชื่อชีทหรือดัชนีชีท)
    update_sheet = sheet.worksheet("sheet1")  # แทน "sheet1" 

    # ลบข้อมูลในชีท (หากต้องการ)
    # update_sheet.clear()

    # อัปเดตข้อมูลลงใน Google Sheets
    gd.set_with_dataframe(update_sheet, filtered_data)

    # พิมพ์เวลาปัจจุบัน
    print(datetime.now())

# เรียกใช้งานฟังก์ชัน run_all()
run_all()


In [ ]:
import schedule
import time

schedule.every().day.at("06:00").do(run_all)

while True:
    schedule.run_pending()
    time.sleep(1)

